In [68]:
import pandas as pd
import numpy as np
import matplotlib_inline as plt
import seaborn as sns
import plotly.express as px
from highcharts_core import highcharts
from highcharts_core.chart import Chart
from highcharts_core.options import HighchartsOptions
from highcharts_core.options.series.dependencywheel import DependencyWheelSeries
from highcharts_core.options.plot_options.dependencywheel import DependencyWheelOptions
from itertools import combinations
from collections import Counter
import plotly.graph_objects as go
from matplotlib import colors



Adjust Pandas' output settings

In [10]:
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)

Load and explore data

In [11]:
df_load = pd.read_parquet('mission_assignments.parquet')
print(df_load.shape)
df_load.head()

(40340, 39)


,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
0,2024081901,R1_Severe Storm-Flooding_08182024,Severe Storm,3612.0,EM,"SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",3612EMCTDOI-USGS01,0,476360,FOS,5.0,High,1,CT,DOI-USGS,DOI-USGS (GEOLOGICAL SURVEY),Stafford Act,2024-08-22T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,2024-08-23T00:00:00.000Z,20000.0,0.0,1.0,0.0,20000.0,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,0.0,1.0,0.0,20000.0,High,USGS Field measurements of flood-water heights...,"As directed by and in coordination with FEMA, ...",ecbcf24e-7d9e-4ced-ae57-4871181703e1,497d57a5e4d071da1721548476b50b64a7321bdd,2024-08-27T18:24:53.439Z
1,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901USDA-APH01,0,475083,FOS,11.0,High,2,PR,USDA-APH,USDA-APHIS (ANIM/PLANT HEALTH,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,29184.0,0.0,1.0,0.0,29184.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,29184.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",72dc30d3-d804-4005-897f-91395be284b2,e0e65997764dc0d660e5b941a18d2f18adabaa55,2024-08-27T18:24:53.439Z
2,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOT01,0,475067,FOS,1.0,High,2,PR,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,25000.0,0.0,1.0,0.0,25000.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,25000.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",61a653ea-688c-4218-bef2-662e1c8b7fde,dbb5a30a3fbd2c96318ba12decb71e63ac072b76,2024-08-27T18:24:53.439Z
3,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOJ-ATF01,0,475098,FOS,13.0,High,2,PR,DOJ-ATF,"DOJ-ATF (ALCOHOL,TOBACCO,FIRE",Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,73500.0,0.0,1.0,0.0,73500.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,73500.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",eeafed44-bdfa-45be-a2ec-6aeb4199043d,4f49b6e6d98634659a77d554c06b8fdeac4503cd,2024-08-27T18:24:53.439Z
4,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOE-OE01,0,475064,FOS,12.0,High,2,PR,DOE-OE,DOE-OE (ELEC DELIVERY / ENERG,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,124822.0,0.0,1.0,0.0,124822.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,124822.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",bdb7935b-5df5-4124-87d8-ddd8feb4a8e3,35d63a525dff8c4a93b83d676d303b6089c0373a,2024-08-27T18:24:53.439Z


In [12]:
# Show only columns missing data as there are too many to show in one cell
missing_data = df_load.isna().sum()
missing_data_ = missing_data[missing_data>0]
missing_data_

disasterNumber         7073
declarationTitle       7073
supportFunction        7508
dateRequired           1853
dateObligated             2
assistanceRequested       2
dtype: int64

Perform a quick review of rows that are missing particular column values.

In [13]:
df_load[df_load['disasterNumber'].isna()].sample(5)

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
192,2024070302,Hurricane Beryl,Hurricane,NaN,SU,None,TX24070302DOT01,0,472190,FOS,1.0,High,6,TX,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2024-07-06T00:00:00.000Z,2024-07-06T00:00:00.000Z,2024-07-06T00:00:00.000Z,2024-07-12T00:00:00.000Z,2024-07-06T00:00:00.000Z,5000.0,0.0,1.0,0.0,5000.0,2024-07-06T00:00:00.000Z,2024-07-10T00:00:00.000Z,0.0,1.0,0.0,5000.0,High,"Activate DOT to RRCC. IOF, JFO, or other teams...","As directed by and in coordination with FEMA, ...",4f0ae679-5098-4b7f-b3de-94fdfe98487f,3580fc6ff64ac1c8048d5cafcb43d3cd9a2cf91a,2024-08-27T18:24:53.439Z
19909,2018090603,Hurricane Florence - DO NOT USE,Hurricane,NaN,SU,None,DC18090603CBP01,1,268936,FOS,0.0,Normal,3,DC,CBP,CUSTOMS AND BORDER PROTECTION,Stafford Act,2018-09-11T00:00:00.000Z,2018-09-11T00:00:00.000Z,2018-09-09T00:00:00.000Z,2018-10-11T00:00:00.000Z,2018-09-12T00:00:00.000Z,-10000.0,0.0,1.0,0.0,-10000.0,2018-09-09T00:00:00.000Z,2018-10-11T00:00:00.000Z,0.0,1.0,0.0,-10000.0,Normal,"Amendment #1: De-obligate $10,000 and close. ...","As directed by and in coordination with FEMA, ...",3f72cbe5-6f0f-47c6-a2c7-afd3116c1eb2,8816d8e1f274a59b62b6407b784ae0295113878c,2024-08-27T18:24:53.439Z
4995,2021082601,Remnants of Hurricane Ida,Hurricane,NaN,SU,None,DC21082601USDA-APH01,2,375844,FOS,11.0,High,3,DC,USDA-APH,USDA-APHIS (ANIM/PLANT HEALTH,Stafford Act,2022-01-19T00:00:00.000Z,2022-01-19T00:00:00.000Z,2021-08-28T00:00:00.000Z,2021-09-11T00:00:00.000Z,2022-01-20T00:00:00.000Z,-6268.0,0.0,1.0,0.0,-6268.0,2021-08-28T00:00:00.000Z,2021-09-11T00:00:00.000Z,0.0,1.0,0.0,-6268.0,High,"Amendment #2: De-obligate ($6,268) per 1/1/202...",\tAs directed by and in coordination with FEMA...,daa8f055-0347-49ff-bd56-2c8cfa4c918e,da465d0d9652c8ffe54033a4fc412c51abb62885,2024-08-27T18:24:53.439Z
40332,2012101103,Region II - TS Rafael,Severe Storm,NaN,SU,None,VI12101103DOD01,1,157029,FOS,NaN,Normal,2,VI,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2012-10-15T00:00:00.000Z,2012-10-15T00:00:00.000Z,2012-10-13T00:00:00.000Z,2012-10-21T00:00:00.000Z,2012-10-16T00:00:00.000Z,-51000.0,0.0,1.0,0.0,-51000.0,2012-10-13T00:00:00.000Z,2012-10-21T00:00:00.000Z,0.0,1.0,0.0,-51000.0,Normal,"Amendment No1: De-obligate $51,000.00, per DCE...","As requested by and in coordination with FEMA,...",d83037b8-6d31-46a7-b83e-6cf98cf6e661,64ed31d687f09efbd89e2c6786881ecce085c827,2024-08-27T18:04:54.403Z
28732,2024092301,Tropical Cyclone Helene,Tropical Storm,NaN,SU,None,GA24092301EPA01,0,479323,FOS,10.0,High,4,GA,EPA,EPA (ENVIRON. PROTECTION AGY),Stafford Act,2024-09-26T00:00:00.000Z,2024-09-25T00:00:00.000Z,2024-09-23T00:00:00.000Z,2024-10-02T00:00:00.000Z,2024-09-26T00:00:00.000Z,83838.0,0.0,1.0,0.0,83838.0,2024-09-23T00:00:00.000Z,2024-10-02T00:00:00.000Z,0.0,1.0,0.0,83838.0,High,"Activate EPA staff to RRCC, IOF, JFO, or other...","As directed by and in coordination with FEMA, ...",37dc3e0b-9392-483f-adf4-d20101598971,69c71e84aea3f669f183f273ac176c680c580641,2024-10-10T16:25:42.309Z


In [14]:
df_load[df_load['supportFunction'].isna()].sample(5)

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
39072,2012102505,Hurricane Sandy,Hurricane,4086.0,DR,HURRICANE SANDY,4086DRNJUSFS01,2,170629,FOS,NaN,High,2,NJ,USFS,U.S. FOREST SERVICE,Stafford Act,2013-09-20T00:00:00.000Z,2013-09-20T00:00:00.000Z,2012-10-31T00:00:00.000Z,2012-12-01T00:00:00.000Z,2013-09-23T00:00:00.000Z,-25570.84,0.0,1.0,0.0,-25570.84,2012-10-31T00:00:00.000Z,2012-12-01T00:00:00.000Z,0.0,1.0,0.0,-25570.84,High,"BOC 2501 Amendment #2 De-obligate $25,570.84 ...",Post-Declaration activation for appropriate US...,60d1fe0d-2157-417a-a58b-c7463b913c98,7e5da3950d3edeae86f20014c23478a4b4feb623,2024-08-27T18:04:54.403Z
40240,2012102504,Hurricane Sandy,Hurricane,NaN,SU,None,NY12102504GSA01,2,163724,FOS,NaN,High,2,NY,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2013-02-12T00:00:00.000Z,2013-02-11T00:00:00.000Z,2012-10-26T00:00:00.000Z,2012-11-01T00:00:00.000Z,2013-02-15T00:00:00.000Z,2506.35,0.0,1.0,0.0,2506.35,2012-10-26T00:00:00.000Z,2012-11-01T00:00:00.000Z,0.0,1.0,0.0,2506.35,High,"Amendment # 2- Increase funds by $2,506.35\r\n...",Pre-Declaration activation for appropriate GSA...,bc56850b-37d5-4529-88b0-2bd5da07d9ea,2ea584b559d62e3b4f22fd8ff271419f56f3e677,2024-08-27T18:04:54.403Z
37513,2015060401,Severe Storms and Flooding,Flood,4227.0,DR,SEVERE STORMS AND FLOODING,4227DRWYDOC01,1,198196,FOS,NaN,High,8,WY,DOC,DEPT. OF COMMERCE,Stafford Act,2015-10-16T00:00:00.000Z,2015-10-16T00:00:00.000Z,2015-10-02T00:00:00.000Z,2015-12-15T00:00:00.000Z,2015-10-22T00:00:00.000Z,15000.00,0.0,1.0,0.0,15000.00,2015-10-02T00:00:00.000Z,2016-10-15T00:00:00.000Z,0.0,1.0,0.0,15000.00,Normal,Amendment #1; Amend to increase funding by $15...,At the direction of FEMA Region VIII and in ac...,d12367d4-5d64-496d-92b0-c48d8ab2f433,49a6331e94965201d0f0b3c4e1573be693dfbd90,2024-08-27T18:04:54.403Z
39703,2012102505,Hurricane Sandy,Hurricane,4085.0,DR,HURRICANE SANDY,4085DRNYIAIP01,3,162376,FOS,NaN,High,2,NY,IAIP,INFO ANALYSIS & INFRAST PROT,Stafford Act,2013-01-08T00:00:00.000Z,2013-01-08T00:00:00.000Z,2012-11-15T00:00:00.000Z,2013-01-31T00:00:00.000Z,2013-01-11T00:00:00.000Z,50000.00,0.0,1.0,0.0,50000.00,2012-11-15T00:00:00.000Z,2013-06-30T00:00:00.000Z,0.0,1.0,0.0,50000.00,High,"Amendment #3: Increase funding by $50,000 to t...",DHS-IP personnel will perform duties in primar...,281a42b5-0761-4f77-bfc1-575fd5fe6176,7954f2c90f32530055409ccc07a37f9ff8c12e27,2024-08-27T18:04:54.403Z
39497,2012102505,Hurricane Sandy,Hurricane,4086.0,DR,HURRICANE SANDY,4086DRNJCOE-NAD02,14,201970,DFA,NaN,Normal,2,NJ,COE-NAD,COE NORTH ATLANTIC DIVISION,Stafford Act,2016-01-15T00:00:00.000Z,2016-01-15T00:00:00.000Z,2012-10-30T00:00:00.000Z,2012-12-05T00:00:00.000Z,2016-02-14T00:00:00.000Z,-7202.75,0.0,1.0,0.0,-7202.75,2012-10-30T00:00:00.000Z,2012-12-05T00:00:00.000Z,0.0,1.0,0.0,-7202.75,Normal,"BOC 2508: Amendment #14: De-obligate $7,202.75...",Deploy Type 2 Emergency Power PRT to include t...,3a2ee57c-59ee-4670-a8d9-9b3ace6fa2d7,51af2ee0118712dd1f8f7a84f1cced7e2985b6ea,2024-08-27T18:04:54.403Z


In [15]:
print(df_load['incidentId'].min(), df_load['incidentId'].max())

2012082201 2025081801


In [16]:
df_load[df_load['incidentId']==2012082201]

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
40337,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,2,165748,FOS,NaN,Normal,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2013-04-23T00:00:00.000Z,2013-04-23T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2013-04-23T00:00:00.000Z,-12279.2,0.0,1.0,0.0,-12279.2,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,-12279.2,Normal,"Amendment #2: De-obligate $12,279.20 per POC/...",Post-Declaration activation for appropriate GS...,43453211-f4c2-40a9-bd65-a28bb2b8d411,c603c54b23b25a28a569ecaea4b89c4420c74c3e,2024-08-27T18:04:54.403Z
40338,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,1,157740,FOS,NaN,Normal,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2012-10-31T00:00:00.000Z,2012-10-30T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2012-10-31T00:00:00.000Z,5000.0,0.0,1.0,0.0,5000.0,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,5000.0,Normal,Amendment # 1 Request additional $5000.00 for ...,Post-Declaration activation for appropriate GS...,b908e133-f794-4112-b48c-c7f82175ede0,465ad5cb1fe3bc2cc61887af16a290311432def0,2024-08-27T18:04:54.403Z
40339,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,0,157223,FOS,NaN,High,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2012-10-22T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2012-10-23T00:00:00.000Z,10000.0,0.0,1.0,0.0,10000.0,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,10000.0,Normal,Activate General Services Administration (GSA)...,Post-Declaration activation for appropriate GS...,93ee3733-615a-4929-9bf4-b2109e1e550d,162eb3cb98ba5b9132b6f939155905bf76c4e8b4,2024-08-27T18:04:54.403Z


Drop rows with no suppportFunction values as we only care about those that do have values.

In [17]:
df = df_load.dropna(subset='supportFunction')

Treat all columns as strings but preserve NaNs.

In [18]:
df = df.astype(str)
for column in df.columns:
    nan_cells = df[column].isna()
    df[column] = df[column].astype(str)
    df.loc[nan_cells, column] = np.nan
df.isna().sum()

incidentId              0
incidentName            0
incidentType            0
disasterNumber          0
declarationType         0
declarationTitle        0
maId                    0
maAmendNumber           0
actionId                0
maType                  0
supportFunction         0
priority                0
region                  0
stt                     0
agencyId                0
agency                  0
authority               0
dateReceived            0
dateRequired            0
popStartDate            0
popEndDate              0
dateObligated           0
obligationAmount        0
sttCostSharePct         0
fedCostSharePct         0
sttCostShareAmt         0
fedCostShareAmt         0
maPopStartDate          0
maPopEndDate            0
maSttCostSharePct       0
maFedCostSharePct       0
maSttCostShareAmount    0
maFedCostShareAmount    0
maPriority              0
assistanceRequested     0
statementOfWork         0
id                      0
hash                    0
lastRefresh 

Display columns in the dataset, verify they are unique

In [19]:
columns = df.columns.to_list()
print(f'There are {len(columns)} in the dataset and {len(set(columns))} are unique')
columns

There are 39 in the dataset and 39 are unique


['incidentId',
 'incidentName',
 'incidentType',
 'disasterNumber',
 'declarationType',
 'declarationTitle',
 'maId',
 'maAmendNumber',
 'actionId',
 'maType',
 'supportFunction',
 'priority',
 'region',
 'stt',
 'agencyId',
 'agency',
 'authority',
 'dateReceived',
 'dateRequired',
 'popStartDate',
 'popEndDate',
 'dateObligated',
 'obligationAmount',
 'sttCostSharePct',
 'fedCostSharePct',
 'sttCostShareAmt',
 'fedCostShareAmt',
 'maPopStartDate',
 'maPopEndDate',
 'maSttCostSharePct',
 'maFedCostSharePct',
 'maSttCostShareAmount',
 'maFedCostShareAmount',
 'maPriority',
 'assistanceRequested',
 'statementOfWork',
 'id',
 'hash',
 'lastRefresh']

Display basic description of dataframe columns

In [20]:
df.describe(include='all')

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
count,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832
unique,440,381,24,586,3,168,9862,33,32832,3,23,4,10,56,144,202,1,3190,3294,1883,2873,3269,13480,4,4,1838,13721,1870,2246,4,4,1303,13717,4,31799,8736,32832,32832,1380
top,2020030901,COVID-19 Pandemic,Hurricane,nan,DR,COVID-19 PANDEMIC,DC20030901DOD15,0,482297,FOS,0.0,High,4,FL,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2024-10-12T00:00:00.000Z,None,2020-08-22T00:00:00.000Z,2021-04-30T00:00:00.000Z,2018-09-12T00:00:00.000Z,0.0,0.0,1.0,0.0,0.0,2020-08-22T00:00:00.000Z,2021-04-30T00:00:00.000Z,0.0,1.0,0.0,0.0,High,"Activate GSA to RRCC, IOF, JFO, or other teams...","As directed by and in coordination with FEMA, ...",d4e38d38-61bb-4ce2-80e9-a07167c925ed,91d92200c7605aca67c6132e336b78b22c210c16,2024-08-27T18:24:53.439Z
freq,8997,8997,11506,5611,23439,8040,33,9582,1,22308,7677,18618,7997,3717,5131,2925,32832,102,1438,1079,655,111,4550,29228,29228,29879,4550,1079,1145,30352,30352,30803,4550,16577,41,74,1,1,27790


Print distribution of values found in specific columns

In [21]:
df['incidentName'].value_counts()

incidentName
COVID-19 Pandemic                                    8997
Hurricane Maria                                      1704
Tropical Cyclone Helene                              1501
Hurricane Florence                                   1025
Remnants of Hurricane Ida                            1001
                                                     ... 
Severe Storms, Straight-line Winds, and Tornadoes       1
AR - Severe Storms - Tornadoes - Flooding               1
SD-Severe Storm - 5-31-2013                             1
AR Severe Storms Tornadoes and Flooding                 1
R5 WI - Aug 2025 - Severe Storms and Flooding           1
Name: count, Length: 381, dtype: int64

In [22]:
df['incidentType'].value_counts()


incidentType
Hurricane              11506
Biological              8997
Tropical Storm          3106
Fire                    2466
Flood                   2066
Severe Storm            1881
Typhoon                  682
Tropical Depression      367
Coastal Storm            363
Earthquake               337
Tornado                  286
Other                    277
Severe Ice Storm         149
Volcanic Eruption        140
Dam/Levee Break           53
Mud/Landslide             48
Snowstorm                 43
Winter Storm              20
Chemical                  18
Freezing                   7
Terrorist                  7
Straight-Line Winds        6
Toxic Substances           4
Tsunami                    3
Name: count, dtype: int64

In [23]:
df['declarationType'].value_counts()

declarationType
DR    23439
SU     5611
EM     3782
Name: count, dtype: int64

In [24]:
df['declarationTitle'].value_counts()

declarationTitle
COVID-19 PANDEMIC                                                8040
None                                                             5611
HURRICANE MARIA                                                  1682
WILDFIRES                                                         829
TROPICAL STORM HELENE                                             806
                                                                 ... 
TROPICAL STORM ERNESTO                                              2
SEVERE STORMS, TORNADO, AND FLOODING                                1
SEVERE STORM AND STRAIGHT-LINE WIND                                 1
SEVERE STORMS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES       1
SEVERE STORMS, STRAIGHT-LINE WINDS, FLOODING, AND MUDSLIDES         1
Name: count, Length: 168, dtype: int64

In [25]:
df['maType'].value_counts()

maType
FOS    22308
DFA    10484
TA        40
Name: count, dtype: int64

In [26]:
df['supportFunction'].value_counts()

supportFunction
0.0     7677
3.0     5716
8.0     3895
7.0     3027
5.0     2147
13.0    1621
10.0    1376
1.0     1073
11.0     925
4.0      904
6.0      848
12.0     821
2.0      682
9.0      459
18.0     335
19.0     310
20.0     240
17.0     197
16.0     167
14.0     162
15.0     153
21.0      54
22.0      43
Name: count, dtype: int64

In [27]:
df['agencyId'].value_counts()

agencyId
DOD         5131
GSA         1890
COE-SAD     1794
HHS         1726
EPA         1538
            ... 
DHS-OPS        1
DOC-BIS        1
OCFO           1
USFWS          1
USDAOCIO       1
Name: count, Length: 144, dtype: int64

In [28]:
df['authority'].value_counts(dropna=False)

authority
Stafford Act    32832
Name: count, dtype: int64

In [29]:
df['assistanceRequested'].value_counts()

assistanceRequested
Activate GSA to RRCC, IOF, JFO, or other teams and facilities as requested                                                       41
USAF Auxiliary, Civil Air Patrol (CAP) aerial support                                                                            22
Activate DOE to RRCC. IOF, JFO, or other team(s) &amp; facilities as requested                                                   20
Activate EPA to RRCC, IOF, JFO, or other teams and facilities as requested                                                       16
Activate USACE to the RRCC, IOF, JFO, or other facilities                                                                        15
                                                                                                                                 ..
Activate VA to NRCC. This is a re-issuance of MA # 3605EM-FL-VA-01 for $3,000.                                                    1
Activate USFS to the NRCC. This is a re-issuance of MA #

In [30]:
df['statementOfWork'].value_counts()

statementOfWork
As directed by and in coordination with FEMA, the US Army Corps of Engineers (USACE) will provide appropriate personnel to the RRCC, IOF, JFO, or other facilities in support of disaster operations. Support may include, but is not limited to, the following: - USACE support to RRCC - USACE support to National and/or Regional IMAT - USACE modeling and/or National Hurricane Program support - USACE Local Government Liaison(s) - USACE support to ESF #15 External Affairs - USACE SME(s) - Infrastructure Systems Recovery Support Function (IS-RSF) Field Coordinator (under NDRF) - Contract or OFA audit support to ensure costs incurred under USACE MAs meet all Stafford Act and regulatory requirements This mission assignment may also fund other site-specific personnel and non-mission specific administrative and management support at a Recovery Field Office (RFO) or other USACE node locations. Those costs are not considered in this initial estimate. Costs would vary based on the magn

In [31]:
df['maPriority'].value_counts()

maPriority
High               16577
Normal             11239
Life Sustaining     2935
Lifesaving          2081
Name: count, dtype: int64

In [32]:
df['region'].value_counts()

region
4     7997
9     6152
2     5081
6     4892
3     2624
1     1635
10    1423
5     1379
7      981
8      668
Name: count, dtype: int64

Plot some information about columns and/or column combinations.

In [33]:
all_states = (df['stt'].unique())
all_states_sorted = sorted(df['stt'].unique())
regions = list(range(1,11))
agency_ids = sorted(df['agencyId'].unique())
incidents = sorted(df['incidentType'].unique())
esfs = sorted(df['supportFunction'].unique())

fig = px.scatter(df.groupby(
        ['agencyId', 'region']).size(       
    ).reset_index(
        name='count').sort_values(by='region'),
                 x = 'agencyId',
                 y = 'region',
                 size = 'count',
                 title = 'Agency by Region',
                 labels = {'agencyId' : 'Agency',
                           'region' : 'Region'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = regions,
    tickmode = 'linear',
    dtick = 1
)

fig.update_layout(
    width = 1600
)

fig.show()

There are definitely some data quality issue with the agencyId values. Those will be addressed later.

In [34]:
fig = px.scatter(df.groupby(
        ['agencyId', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='incidentType'),
                 x = 'agencyId',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Agency by Incident Type',
                 labels = {'agencyId' : 'Agency',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1
)

fig.update_layout(
    height = 800,
    width = 1600
)

fig.show()

In [35]:
fig = px.scatter(df.groupby(
        ['supportFunction', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='incidentType'),
                 x = 'supportFunction',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Support Function by Incident Type',
                 labels = {'supportFunction' : 'Support Function',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = esfs,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1
)
fig.update_layout(
    height = 800,
    width = 800
)

fig.show()

In [36]:
fig = px.scatter(df.groupby(
        ['supportFunction', 'agencyId']).size(       
    ).reset_index(
        name='count').sort_values(by='agencyId'),
                 x = 'supportFunction',
                 y = 'agencyId',
                 size = 'count',
                 title = 'Support Function by Incident Type',
                 labels = {'supportFunction' : 'Support Function',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = esfs,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)
fig.update_layout(
    height = 1600,
    width = 800
)

fig.show()

In [37]:
df['year'] = df['incidentId'].str[:4]
df.head()

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh,year
0,2024081901,R1_Severe Storm-Flooding_08182024,Severe Storm,3612.0,EM,"SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",3612EMCTDOI-USGS01,0,476360,FOS,5.0,High,1,CT,DOI-USGS,DOI-USGS (GEOLOGICAL SURVEY),Stafford Act,2024-08-22T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,2024-08-23T00:00:00.000Z,20000.0,0.0,1.0,0.0,20000.0,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,0.0,1.0,0.0,20000.0,High,USGS Field measurements of flood-water heights...,"As directed by and in coordination with FEMA, ...",ecbcf24e-7d9e-4ced-ae57-4871181703e1,497d57a5e4d071da1721548476b50b64a7321bdd,2024-08-27T18:24:53.439Z,2024
1,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901USDA-APH01,0,475083,FOS,11.0,High,2,PR,USDA-APH,USDA-APHIS (ANIM/PLANT HEALTH,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,29184.0,0.0,1.0,0.0,29184.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,29184.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",72dc30d3-d804-4005-897f-91395be284b2,e0e65997764dc0d660e5b941a18d2f18adabaa55,2024-08-27T18:24:53.439Z,2024
2,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOT01,0,475067,FOS,1.0,High,2,PR,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,25000.0,0.0,1.0,0.0,25000.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,25000.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",61a653ea-688c-4218-bef2-662e1c8b7fde,dbb5a30a3fbd2c96318ba12decb71e63ac072b76,2024-08-27T18:24:53.439Z,2024
3,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOJ-ATF01,0,475098,FOS,13.0,High,2,PR,DOJ-ATF,"DOJ-ATF (ALCOHOL,TOBACCO,FIRE",Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,73500.0,0.0,1.0,0.0,73500.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,73500.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",eeafed44-bdfa-45be-a2ec-6aeb4199043d,4f49b6e6d98634659a77d554c06b8fdeac4503cd,2024-08-27T18:24:53.439Z,2024
4,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOE-OE01,0,475064,FOS,12.0,High,2,PR,DOE-OE,DOE-OE (ELEC DELIVERY / ENERG,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,124822.0,0.0,1.0,0.0,124822.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,124822.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",bdb7935b-5df5-4124-87d8-ddd8feb4a8e3,35d63a525dff8c4a93b83d676d303b6089c0373a,2024-08-27T18:24:53.439Z,2024


In [38]:
df['incidentType'].unique()

array(['Severe Storm', 'Tropical Storm', 'Fire', 'Hurricane', 'Flood',
       'Tornado', 'Other', 'Biological', 'Winter Storm', 'Snowstorm',
       'Volcanic Eruption', 'Coastal Storm', 'Severe Ice Storm',
       'Dam/Levee Break', 'Typhoon', 'Earthquake', 'Freezing',
       'Mud/Landslide', 'Toxic Substances', 'Chemical', 'Terrorist',
       'Tsunami', 'Tropical Depression', 'Straight-Line Winds'],
      dtype=object)

In [39]:
fig = px.scatter(df.groupby(
        ['year', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='year'),
                 x = 'year',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Incident Type by year',
                 labels = {'year' : 'Year',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = sorted(df['year'].unique()),
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1
)
fig.update_layout(
    height = 800,
    width = 800
)
fig.show()

We want to get an idea of how many times a supportFunction value appears with others in the same incidentId. Our first attempt will be utilizing a Dependency Wheel chart from Highcharts.

Note: This library does not always display properly, so the output is to a .png file instead.

In [57]:
# Create supportFunction pairs

pair_counts = {}

for incident_id, group in df.groupby("incidentId"):
    funcs = group["supportFunction"].dropna().unique()

    for a, b in combinations(sorted(funcs), 2):
        pair_counts[(a, b)] = pair_counts.get((a, b), 0) + 1

out = (
    pd.DataFrame(
        [(a, b, count) for (a, b), count in pair_counts.items()],
        columns=["supportFunction_A", "supportFunction_B", "count"]
    )
    .sort_values(["supportFunction_A", "supportFunction_B"])
    .reset_index(drop=True)
)

print(out)

results_list = out.values.tolist()

cleaned = [
    {"from": str(a), "to": str(b), "weight": int(w)}
    for a, b, w in results_list
]

series = DependencyWheelSeries(
    name="Support Function Links",
    data=cleaned,
    keys=["from", "to", "weight"]
)

plot_opts = DependencyWheelOptions(
    node_padding=12,
    node_width=25
)

options = HighchartsOptions(
    chart={"type": "dependencywheel"},
    title={"text": "Support Function Dependency Wheel"},
    plot_options={"dependencywheel": plot_opts},
    series=[series]
)

# Wrap in Chart object
chart = Chart.from_options(options)

chart.download_chart(filename='esf_incidentType_wheel.png')


    supportFunction_A supportFunction_B  count
0                 0.0               1.0     95
1                 0.0              10.0     86
2                 0.0              11.0     74
3                 0.0              12.0     84
4                 0.0              13.0     93
..                ...               ...    ...
247               6.0               8.0     49
248               6.0               9.0     26
249               7.0               8.0    121
250               7.0               9.0     42
251               8.0               9.0     47

[252 rows x 3 columns]


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01\x90\x08\x02\x00\x00\x00\xfdW\x89\xcf\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x01\xec\xbd\x07|\x94\xc7\x99?\xae\x9c\xe3\x0biw\xfe_\\.\x97K\xb9\xdc\xdd\xdf\xbf\xbb\xcb/q\xb2\xb1\xc1\xe0\x86\xc1%\x87\x1d\x17\\\xc0=\xb6\xb1\x1d\x97\x80\x89\xc1\x050\xa6\xd8\xc46\x06\x8c\x11M\x80$$$\xadV}W\xd2\xaa\xb7\xd5\xaa\xf7\xb2E[\xb4\xbdj{\xd1\x96w\xe6\xe7wg5\xfb\xb2\x92v\x05\x08\x1b\xac\xd1g?\xd2\xbc\xef\xcc;\xe5;\xaf\xe6\xbb\xcf3\xcf\xf3L\n$?\x04\x01\x82\x00A\x80 @\x10X\xc0\x08\xa4,\xe0\xb1\x93\xa1\x13\x04\x08\x02\x04\x01\x82\x00A\x00\x12"$/\x01A\x80 @\x10 \x08,h\x04\x08\x11.\xe8\xe9\'\x83\'\x08\x10\x04\x08\x02\x04\x01B\x84\xe4\x1d \x08\x10\x04\x08\x02\x04\x81\x05\x8d\x00!\xc2\x05=\xfdd\xf0\x04\x01\x82\x00A\x80 @\x88\x90\xbc\x03\x04\x01\x82\x00A\x80 \xb0\xa0\x11 D\xb8\xa0\xa7\x9f\x0c\x9e @\x10 \x08\x10\x04\x08\x11\x92w\x80 @\x10 \x08\x10\x04\x164\x02\x84\x08\x17\xf4\xf4\x93\xc1\x13\x04\x08\x02\x04\x01\x82\x00!B\xf2\x0e\x

The next plot is a circular Sankey chart using Plotly, which is not quite as successful a visualization but does appear in line. This was done with the help of Chat-GPT.

In [76]:
pairs = []
for incident, group in df.groupby("incidentId"):
    funcs = group["supportFunction"].unique()
    pairs.extend(combinations(sorted(funcs), 2))

pair_counts = Counter(pairs)

nodes = sorted(set([f for pair in pair_counts.keys() for f in pair]))
num_nodes = len(nodes)
angles = np.linspace(0, 2*np.pi, num_nodes, endpoint=False)
radius = 1
node_coords = {node: (radius*np.cos(a), radius*np.sin(a)) for node, a in zip(nodes, angles)}
node_counts = df['supportFunction'].value_counts().to_dict()
min_size = 10
max_size = 40

def size_for_count(count):
    min_count = min(node_counts.values())
    max_count = max(node_counts.values())
    if max_count == min_count:
        return (min_size + max_size) / 2
    # linear scaling
    return min_size + (count - min_count) / (max_count - min_count) * (max_size - min_size)

node_sizes = [size_for_count(node_counts[node]) for node in nodes]

max_count = max(pair_counts.values())
min_count = min(pair_counts.values())

def color_for_value(value):
    # Linear interpolation from light gray to red to black
    cmap = colors.LinearSegmentedColormap.from_list("", ["lightgray", "red", "black"])
    norm = (value - min_count) / (max_count - min_count) if max_count != min_count else 0
    rgb = cmap(norm)[:3]  # RGB tuple 0-1
    return f'rgb({int(rgb[0]*255)}, {int(rgb[1]*255)}, {int(rgb[2]*255)})'

fig = go.Figure()

# Add links
for (a, b), value in pair_counts.items():
    x0, y0 = node_coords[a]
    x1, y1 = node_coords[b]
    fig.add_trace(go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        mode='lines',
        line=dict(width=2, color=color_for_value(value)),  # fixed width, color shows weight
        hoverinfo='text',
        text=f"{a} → {b}: {value}",
        showlegend=False
    ))

# Add nodes
fig.add_trace(go.Scatter(
    x=[coord[0] for coord in node_coords.values()],
    y=[coord[1] for coord in node_coords.values()],
    mode='markers+text',
    text=nodes,
    textposition="top center",
    marker=dict(size=node_sizes, color='orange'),
    hoverinfo='text',
    showlegend=False
))

# Layout adjustments
fig.update_layout(
    title="Support Function Dependency Wheel (Links Coded Gray to Red to Black)",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    width=700,
    height=700,
    plot_bgcolor="white"
)

fig.show()
